## Imports

In [1]:
# instalar as dependências
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()

In [4]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [5]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Carregamento de Dados
---

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .getOrCreate()

In [7]:
spark

### Montando o drive

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Carregando os dados dos Resultados

In [9]:
path = '/content/drive/MyDrive/resultados/SP_turno_1.csv'
resultados = spark.read.csv(path, sep=';', inferSchema=True, header=True, encoding='ISO-8859-1')

### Carregando os dados do Eleitorado

In [10]:
path = '/content/drive/MyDrive/eleitorado/perfil_eleitorado_2020/perfil_eleitorado_2020.csv'
eleitorado = spark.read.csv(path, sep=';', inferSchema=True, header=True, encoding='ISO-8859-1')

# Manipulando os Dados
---

## Operações básicas

In [16]:
resultados.limit(25).toPandas()

,NM_MUNICIPIO,NR_ZONA,NR_SECAO,DS_CARGO_PERGUNTA,SG_PARTIDO,QT_APTOS,QT_COMPARECIMENTO,QT_ABSTENCOES,NM_VOTAVEL,QT_VOTOS
0,SÃO PAULO,1,1,Prefeito,#NULO#,386,241,145,Branco,5
1,SÃO PAULO,1,1,Prefeito,PT,386,241,145,JILMAR TATTO,4
2,SÃO PAULO,1,1,Prefeito,REDE,386,241,145,MARINA HELOU,1
3,SÃO PAULO,1,1,Prefeito,PSB,386,241,145,MÁRCIO FRANÇA,29
4,SÃO PAULO,1,1,Prefeito,PSDB,386,241,145,BRUNO COVAS,111
5,SÃO PAULO,1,1,Prefeito,PSL,386,241,145,JOICE HASSELMANN,2
6,SÃO PAULO,1,1,Prefeito,PATRIOTA,386,241,145,ARTHUR DO VAL MAMÃE FALEI,10
7,SÃO PAULO,1,1,Prefeito,#NULO#,386,241,145,Nulo,10
8,SÃO PAULO,1,1,Prefeito,REPUBLICANOS,386,241,145,CELSO RUSSOMANNO,18
9,SÃO PAULO,1,1,Prefeito,PSOL,386,241,145,GUILHERME BOULOS,43


In [14]:
eleitorado.limit(5).toPandas()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,SG_UF,CD_MUNICIPIO,NM_MUNICIPIO,CD_MUN_SIT_BIOMETRIA,DS_MUN_SIT_BIOMETRIA,NR_ZONA,CD_GENERO,DS_GENERO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_FAIXA_ETARIA,DS_FAIXA_ETARIA,CD_GRAU_ESCOLARIDADE,DS_GRAU_ESCOLARIDADE,QT_ELEITORES_PERFIL,QT_ELEITORES_BIOMETRIA,QT_ELEITORES_DEFICIENCIA,QT_ELEITORES_INC_NM_SOCIAL
0,01/08/2020,08:37:48,2020,SP,70734,SÃO BENTO DO SAPUCAÍ,0,Sem biometria,314,4,FEMININO,3,CASADO,7074,70 a 74 anos,3,ENSINO FUNDAMENTAL INCOMPLETO,47,37,1,0
1,01/08/2020,08:37:48,2020,SP,70734,SÃO BENTO DO SAPUCAÍ,0,Sem biometria,314,4,FEMININO,3,CASADO,7074,70 a 74 anos,4,ENSINO FUNDAMENTAL COMPLETO,5,4,0,0
2,01/08/2020,08:37:48,2020,SP,70734,SÃO BENTO DO SAPUCAÍ,0,Sem biometria,314,4,FEMININO,3,CASADO,7074,70 a 74 anos,5,ENSINO MÉDIO INCOMPLETO,1,1,0,0
3,01/08/2020,08:37:48,2020,SP,70734,SÃO BENTO DO SAPUCAÍ,0,Sem biometria,314,4,FEMININO,3,CASADO,7074,70 a 74 anos,6,ENSINO MÉDIO COMPLETO,10,8,0,0
4,01/08/2020,08:37:48,2020,SP,70734,SÃO BENTO DO SAPUCAÍ,0,Sem biometria,314,4,FEMININO,3,CASADO,7074,70 a 74 anos,8,SUPERIOR COMPLETO,23,18,0,0


### Selecionando as colunas do DataFrame

In [11]:
resultados

DataFrame[DT_GERACAO: string, HH_GERACAO: string, ANO_ELEICAO: int, CD_TIPO_ELEICAO: int, NM_TIPO_ELEICAO: string, CD_PLEITO: int, DT_PLEITO: string, NR_TURNO: int, CD_ELEICAO: int, DS_ELEICAO: string, SG_UF: string, CD_MUNICIPIO: int, NM_MUNICIPIO: string, NR_ZONA: int, NR_SECAO: int, NR_LOCAL_VOTACAO: int, CD_CARGO_PERGUNTA: int, DS_CARGO_PERGUNTA: string, NR_PARTIDO: int, SG_PARTIDO: string, NM_PARTIDO: string, DT_BU_RECEBIDO: string, QT_APTOS: int, QT_COMPARECIMENTO: int, QT_ABSTENCOES: int, CD_TIPO_URNA: int, DS_TIPO_URNA: string, CD_TIPO_VOTAVEL: int, DS_TIPO_VOTAVEL: string, NR_VOTAVEL: int, NM_VOTAVEL: string, QT_VOTOS: int, NR_URNA_EFETIVADA: int, CD_CARGA_1_URNA_EFETIVADA: string, CD_CARGA_2_URNA_EFETIVADA: double, CD_FLASHCARD_URNA_EFETIVADA: string, DT_CARGA_URNA_EFETIVADA: string, DS_CARGO_PERGUNTA_SECAO: string, DS_AGREGADAS: string, DT_ABERTURA: string, DT_ENCERRAMENTO: string, QT_ELEITORES_BIOMETRIA_NH: int, DT_EMISSAO_BU: string, NR_JUNTA_APURADORA: int, NR_TURMA_APURA

In [12]:
eleitorado

DataFrame[DT_GERACAO: string, HH_GERACAO: string, ANO_ELEICAO: int, SG_UF: string, CD_MUNICIPIO: int, NM_MUNICIPIO: string, CD_MUN_SIT_BIOMETRIA: int, DS_MUN_SIT_BIOMETRIA: string, NR_ZONA: int, CD_GENERO: int, DS_GENERO: string, CD_ESTADO_CIVIL: int, DS_ESTADO_CIVIL: string, CD_FAIXA_ETARIA: int, DS_FAIXA_ETARIA: string, CD_GRAU_ESCOLARIDADE: int, DS_GRAU_ESCOLARIDADE: string, QT_ELEITORES_PERFIL: int, QT_ELEITORES_BIOMETRIA: int, QT_ELEITORES_DEFICIENCIA: int, QT_ELEITORES_INC_NM_SOCIAL: int]

In [11]:
resultados = resultados.select("NM_MUNICIPIO", "NR_ZONA", "NR_SECAO", "DS_CARGO_PERGUNTA", "SG_PARTIDO", "QT_APTOS", "QT_COMPARECIMENTO", "QT_ABSTENCOES", "NM_VOTAVEL", "QT_VOTOS")

In [12]:
eleitorado = eleitorado.select("NM_MUNICIPIO", "NR_ZONA", "DS_GENERO", "DS_ESTADO_CIVIL", "DS_FAIXA_ETARIA", "DS_GRAU_ESCOLARIDADE", "QT_ELEITORES_PERFIL", "QT_ELEITORES_DEFICIENCIA", "QT_ELEITORES_INC_NM_SOCIAL")

## Encontrando o prefeito mais votado de cada cidade

In [128]:
# Aqui, estamos agrupando os resultados com base em múltiplas colunas:
# Para cada grupo, estamos somando os votos ("QT_VOTOS") e renomeando essa soma para "TOTAL_VOTOS".
candidato_mais_votado = resultados.groupBy("NM_MUNICIPIO", "NM_VOTAVEL", "DS_CARGO_PERGUNTA", "SG_PARTIDO").agg(F.sum("QT_VOTOS").alias("TOTAL_VOTOS"))

# Nesta linha, estamos filtrando apenas os registros onde o cargo em disputa é "Prefeito".
candidato_mais_votado_prefeito = candidato_mais_votado.filter(candidato_mais_votado["DS_CARGO_PERGUNTA"] == "Prefeito")

# Aqui, estamos definindo uma janela de análise (window) que particiona os dados pelo "NM_MUNICIPIO" (nome do município)
# e os ordena em ordem decrescente pelo "TOTAL_VOTOS" (quantidade de votos).
window_spec = Window.partitionBy("NM_MUNICIPIO").orderBy(F.desc("TOTAL_VOTOS"))

# Com base na janela definida acima, estamos adicionando uma nova coluna chamada "rank" ao DataFrame.
# Esta coluna "rank" representa o ranking de cada candidato a prefeito em seu respectivo município com base no número total de votos.
candidato_mais_votado_prefeito_ranked = candidato_mais_votado_prefeito.withColumn("rank", F.row_number().over(window_spec))

# Finalmente, filtramos apenas os candidatos que estão em 1º lugar no ranking (ou seja, os mais votados) em cada município.
candidato_mais_votado_in_each_municipio_prefeito = candidato_mais_votado_prefeito_ranked.filter(candidato_mais_votado_prefeito_ranked["rank"] == 1)


In [129]:
candidato_mais_votado_in_each_municipio_prefeito.toPandas()

,NM_MUNICIPIO,NM_VOTAVEL,DS_CARGO_PERGUNTA,SG_PARTIDO,TOTAL_VOTOS,rank
0,CANAS,SILVANA ZANIN,Prefeito,PDT,1108,1
1,PORTO FELIZ,DR CÁSSIO,Prefeito,PTB,25318,1
2,RIBEIRÃO BRANCO,MAURO TEIXEIRA,Prefeito,PP,5914,1
3,SÃO JOAQUIM DA BARRA,SCHMIDT,Prefeito,MDB,9789,1
4,ARCO-ÍRIS,ALDO MANSANO,Prefeito,PV,1134,1
...,...,...,...,...,...,...
640,SÃO SEBASTIÃO DA GRAMA,ZÉ DA DOCA,Prefeito,PTB,2402,1
641,NOVA CANAÃ PAULISTA,THAIS,Prefeito,PSDB,743,1
642,CAMPINA DO MONTE ALEGRE,ZE DITO,Prefeito,PSDB,2439,1
643,SÃO JOSÉ DO BARREIRO,LÊ BRAGA,Prefeito,PSDB,1570,1


## Buscando a porcentagem de votos que um candidato teve em cada zona e informações do perfil de eleitor predominante nela.

In [12]:
def votos_candidato_zona(nome_candidato, partido, df_votos, df_eleitorado):
    # Filtrar o dataframe de votos pelo nome do candidato e partido
    df_candidato_votos = df_votos.filter((df_votos.NM_VOTAVEL == nome_candidato) & (df_votos.SG_PARTIDO == partido))

    # Agrupar pelo NM_MUNICIPIO e NR_ZONA para somar a quantidade de votos por zona
    df_candidato_votos_agrupado = df_candidato_votos.groupBy("NM_MUNICIPIO", "NR_ZONA").agg(F.sum("QT_VOTOS").alias("TOTAL_VOTOS"))

    # Ordenar as zonas pelo número de votos em ordem decrescente
    df_candidato_votos_agrupado = df_candidato_votos_agrupado.orderBy("TOTAL_VOTOS", ascending=False)

    # Juntar o dataframe de eleitorado com o dataframe de votos agrupados usando o NM_MUNICIPIO e NR_ZONA
    df_joined = df_candidato_votos_agrupado.join(df_eleitorado, ["NM_MUNICIPIO", "NR_ZONA"])

    # Computa a soma dos perfis para cada categoria
    df_sum = df_joined.groupBy("NM_MUNICIPIO", "NR_ZONA", "DS_GENERO", "DS_FAIXA_ETARIA", "DS_GRAU_ESCOLARIDADE").agg(F.sum("QT_ELEITORES_PERFIL").alias("TOTAL_PERFIL"))

    # Encontrar a moda das colunas de interesse
    window = Window.partitionBy("NM_MUNICIPIO", "NR_ZONA").orderBy(F.desc("TOTAL_PERFIL"))

    df_genero = df_sum.withColumn("rank", F.row_number().over(window)).filter("rank = 1").select("NM_MUNICIPIO", "NR_ZONA", "DS_GENERO").drop("rank")
    df_faixa = df_sum.withColumn("rank", F.row_number().over(window)).filter("rank = 1").select("NM_MUNICIPIO", "NR_ZONA", "DS_FAIXA_ETARIA").drop("rank")
    df_escolaridade = df_sum.withColumn("rank", F.row_number().over(window)).filter("rank = 1").select("NM_MUNICIPIO", "NR_ZONA", "DS_GRAU_ESCOLARIDADE").drop("rank")

    # Agregar o total de eleitores por NM_MUNICIPIO e NR_ZONA
    df_total_eleitores = df_eleitorado.groupBy("NM_MUNICIPIO", "NR_ZONA").agg(F.sum("QT_ELEITORES_PERFIL").alias("TOTAL_ELEITORES"))

    # Calcular a porcentagem de votos
    df_candidato_votos_agrupado = df_candidato_votos_agrupado.join(df_total_eleitores, ["NM_MUNICIPIO", "NR_ZONA"])
    df_candidato_votos_agrupado = df_candidato_votos_agrupado.withColumn("PERCENTUAL_VOTOS", F.round((F.col("TOTAL_VOTOS") / F.col("TOTAL_ELEITORES")) * 100, 2))
    # Junta os resultados
    df_result = df_candidato_votos_agrupado.join(df_genero, ["NM_MUNICIPIO", "NR_ZONA"]).join(df_faixa, ["NM_MUNICIPIO", "NR_ZONA"]).join(df_escolaridade, ["NM_MUNICIPIO", "NR_ZONA"])

    return df_result

In [16]:
result = votos_candidato_zona('GUILHERME BOULOS', 'PSOL', resultados, eleitorado)
result_pandas = result.toPandas()

In [17]:
result_pandas

,NM_MUNICIPIO,NR_ZONA,TOTAL_VOTOS,TOTAL_ELEITORES,PERCENTUAL_VOTOS,DS_GENERO,DS_FAIXA_ETARIA,DS_GRAU_ESCOLARIDADE
0,SÃO PAULO,327,15166,116314,13.04,FEMININO,35 a 39 anos,ENSINO MÉDIO COMPLETO
1,SÃO PAULO,398,10248,99098,10.34,FEMININO,30 a 34 anos,ENSINO MÉDIO COMPLETO
2,SÃO PAULO,247,12863,107948,11.92,FEMININO,35 a 39 anos,ENSINO MÉDIO COMPLETO
3,SÃO PAULO,250,22095,165668,13.34,FEMININO,40 a 44 anos,SUPERIOR COMPLETO
4,SÃO PAULO,20,21497,157859,13.62,FEMININO,35 a 39 anos,ENSINO MÉDIO COMPLETO
5,SÃO PAULO,254,13977,153234,9.12,FEMININO,35 a 39 anos,ENSINO MÉDIO COMPLETO
6,SÃO PAULO,352,19298,203201,9.50,FEMININO,35 a 39 anos,ENSINO MÉDIO COMPLETO
7,SÃO PAULO,6,23573,159499,14.78,FEMININO,40 a 44 anos,SUPERIOR COMPLETO
8,SÃO PAULO,258,19835,185209,10.71,FEMININO,40 a 44 anos,SUPERIOR COMPLETO
9,SÃO PAULO,373,16707,128866,12.96,FEMININO,35 a 39 anos,ENSINO MÉDIO COMPLETO


In [36]:
result_pandas.to_csv('ResultadosRLPODE2020.csv', sep=';', encoding='ISO-8859-1')

## Buscar zonas eleitorais com maior número de abstenções e votos brancos e nulos

In [24]:
def analyze_by_cargo_and_municipio(df, cargo, municipio=None):
    # Filtrar dados para o cargo especificado
    df_filtered = df.filter(df["DS_CARGO_PERGUNTA"] == cargo)

    # Se um município foi especificado, filtrar os dados para esse município
    if municipio:
        df_filtered = df_filtered.filter(df["NM_MUNICIPIO"] == municipio)

    # Agrupar por NR_ZONA e NR_SECAO para pegar a quantidade única de abstenções em cada seção
    grouped_secao_df = df_filtered.groupBy("NR_ZONA", "NR_SECAO").agg(
        F.first("QT_ABSTENCOES").alias("QT_ABSTENCOES"),
        F.first("QT_APTOS").alias("QT_APTOS")
    )

    # Agrupar por NR_ZONA para somar as abstenções e eleitores aptos em cada zona
    grouped_zona_df = grouped_secao_df.groupBy("NR_ZONA").agg(
        F.sum("QT_ABSTENCOES").alias("Número de abstenções"),
        F.sum("QT_APTOS").alias("Número de aptos")
    )

    # Filtrar linhas onde NM_VOTAVEL é 'Voto Branco' ou 'Voto Nulo'
    branco_nulo_df = df_filtered.filter(df["NM_VOTAVEL"].isin(["Branco", "Nulo"]))

    # Agrupar por NR_ZONA para somar os votos brancos e nulos
    branco_nulo_grouped = branco_nulo_df.groupBy("NR_ZONA").agg(
        F.sum("QT_VOTOS").alias("Número de votos brancos e nulos")
    )

    # Unir os dois DataFrames agrupados para ter todas as informações em um único DataFrame
    final_df = grouped_zona_df.join(branco_nulo_grouped, "NR_ZONA", "left_outer")

    # Calcular a soma de abstenções, votos brancos e nulos
    final_df = final_df.withColumn("Total", final_df["Número de abstenções"] + final_df["Número de votos brancos e nulos"])

    # Calcular a porcentagem de abstenções e votos brancos e nulos em relação aos eleitores aptos
    final_df = final_df.withColumn("Percentual", F.round((final_df["Total"] / final_df["Número de aptos"]) * 100, 2))


    # Ordenar pelo total e pegar as 50 zonas com os maiores números
    top_50_zonas = final_df.orderBy(F.desc("Total")).limit(50)

    return top_50_zonas


# Exemplos de uso
top_50_prefeito_sao_paulo = analyze_by_cargo_and_municipio(resultados, "Prefeito", "SÃO PAULO")
top_50_vereador_sao_paulo = analyze_by_cargo_and_municipio(resultados, "Vereador", "SÃO PAULO")
top_50_prefeito_campinas = analyze_by_cargo_and_municipio(resultados, "Prefeito", "CAMPINAS")
top_50_vereador_campinas = analyze_by_cargo_and_municipio(resultados, "Vereador", "CAMPINAS")

# Mostrar os resultados
print("Top 50 Zonas para Prefeito em São Paulo:")
top_50_prefeito_sao_paulo.show()
print("Top 50 Zonas para Vereador em São Paulo:")
top_50_vereador_sao_paulo.show()

print("Top 50 Zonas para Prefeito em Campinas:")
top_50_prefeito_campinas.show()
print("Top 50 Zonas para Vereador em Campinas:")
top_50_vereador_campinas.show()

Top 50 Zonas para Prefeito em São Paulo:
+-------+--------------------+---------------+-------------------------------+------+----------+
|NR_ZONA|Número de abstenções|Número de aptos|Número de votos brancos e nulos| Total|Percentual|
+-------+--------------------+---------------+-------------------------------+------+----------+
|    372|               70391|         267693|                          37388|107779|     40.26|
|    375|               67245|         245813|                          33561|100806|     41.01|
|    376|               68094|         239785|                          31648| 99742|      41.6|
|    374|               69775|         239916|                          24711| 94486|     39.38|
|    381|               56287|         226777|                          33285| 89572|      39.5|
|    253|               64822|         217680|                          21479| 86301|     39.65|
|    352|               54210|         203182|                          30294| 84504| 